I have a csv file with all the date/times of ASTER observations I'm using, and I want to now download all the MODIS (MOD021KM) imagery for the same date/times covering the same spot.

I'm using the Earthdata API because doing this through their web portal would be a slow and cumbersome process.

Some references:
 * [Earthdata API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html)
 * [MOD021KM product](https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/MOD021KM/)

In [1]:
import requests
import json
import pandas as pd
import io

Read in ASTER timestamps file

In [2]:
aster = pd.read_csv('aster_AST_L1T_band14_Tuolumne-and-CUES.csv')

Set up some of the granule search criteria for Earthdata search API

In [3]:
url = "https://cmr.earthdata.nasa.gov/search/granules.csv?" #"https://cmr.earthdata.nasa.gov/search/collections?has_granules_created_at\[\]=2015-01-01T19:00:00Z,"

shortname_id = "short_name=MOD021KM"
version = "version=6.1"

# bounding box around the Tuolumne area
# lower left longitude, lower left latitude, upper right longitude, upper right latitude
bounding_box = "bounding_box[]=-120,37,118,38" 

In [4]:
# empty list to hold modis download URLs
modis_urls = []

for i, this_timestamp in enumerate(aster.timestampUTC):
       
    # the date is the first 10 characters of the timestamp string
    date = this_timestamp[:10]
    #print(date)
    
    
    # Terra ASTER/MODIS is always flying over here at ~18:50 UTC (day) or ~5:50 UTC (night)
    hours = [(18, 19), (5, 6)]
    for h in hours:
        start_time = "{date}T{hour_start}:00:00Z".format(date=date,hour_start=h[0])
        end_time = "{date}T{hour_end}:00:00Z".format(date=date,hour_end=h[1])
        time_range = "temporal=" + start_time + "," + end_time
    
        # build the whole request URL and make the request.get
        response = requests.get(url+"&"+shortname_id+"&"+version+"&"+bounding_box+"&"+time_range)
    
        # read the response CSV and put in a temporary dataframe
        df = pd.read_csv(io.StringIO(response.text))
        
        for modis_url in df['Online Access URLs']:
            modis_urls.append(modis_url)

Write the list of URLs out to a text file:

In [5]:
with open("modis_download_list.txt", "w") as output:
    for row in modis_urls:
        output.write(str(row) + '\n')